# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 26

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
metric = 'valuePoints metric'
#metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
603,Reina,Aston Villa,45,4.000000,16,18.190494,4.0,4.547624,5.864714,5.500076,2.143770
411,Patrício,Wolves,51,25.897436,101,115.550797,3.9,4.461862,5.541906,4.880387,1.975746
341,Gazzaniga,Tottenham,43,16.764706,57,66.928413,3.4,3.992221,8.251391,5.114037,1.925220
340,Lloris,Tottenham,53,10.000000,42,43.978352,4.2,4.397835,5.706884,5.717816,1.910298
471,Henderson,Sheffield Utd,51,25.111111,113,104.732037,4.5,4.170745,5.824877,5.232335,1.846837
93,Pope,Burnley,47,26.097561,107,103.289031,4.1,3.957804,5.181989,4.919760,1.825599
494,Ramsdale,Bournemouth,46,24.722222,89,92.355396,3.6,3.735724,3.550591,3.575423,1.741790
47,Ryan,Brighton,48,26.000000,91,97.144051,3.5,3.736310,3.308505,3.469520,1.705384
14,Leno,Arsenal,50,25.675676,95,95.843786,3.7,3.732863,3.245389,3.618090,1.669387


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
297,Lundstram,Sheffield Utd,50,25.000000,115,112.620353,4.6,4.504814,2.827409,3.058741,2.014614
182,Alexander-Arnold,Liverpool,77,25.932203,153,144.130662,5.9,5.557980,5.388696,5.343795,2.002957
181,Robertson,Liverpool,70,25.882353,132,129.891261,5.1,5.018526,5.231315,5.054216,1.896825
208,Otamendi,Manchester City,50,16.842105,64,71.290649,3.8,4.232882,5.110846,4.464545,1.893003
185,Matip,Liverpool,52,6.956522,32,29.797195,4.6,4.283347,NaN,NaN,1.878372
105,Azpilicueta,Chelsea,58,23.939394,79,105.499730,3.3,4.406951,4.612068,5.708263,1.829887
405,Boly,Wolves,47,10.000000,46,39.496528,4.6,3.949653,3.557037,3.556140,1.821839
103,Alonso,Chelsea,60,7.966102,47,35.066866,5.9,4.402011,NaN,6.851449,1.797113
440,Angeliño,Manchester City,47,6.111111,11,23.372791,1.8,3.824638,NaN,NaN,1.764174


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,128,23.013699,168,162.287922,7.3,7.051797,7.683430,7.778164,1.971037
215,De Bruyne,Manchester City,108,25.000000,175,156.276740,7.0,6.251070,6.943261,6.144492,1.902140
431,Pulisic,Chelsea,69,15.909091,70,72.990496,4.4,4.587974,NaN,2.665095,1.746612
192,Mané,Liverpool,122,23.030303,152,139.255400,6.6,6.046616,2.571443,4.236628,1.731141
29,Grealish,Aston Villa,67,23.958333,115,104.674075,4.8,4.369005,5.057142,4.733735,1.687894
417,Traoré,Wolves,58,24.761905,104,100.389151,4.2,4.054177,3.584061,3.986890,1.683405
287,Cantwell,Norwich City,49,26.111111,94,94.933604,3.6,3.635755,2.444315,3.176994,1.642467
301,Fleck,Sheffield Utd,51,22.820513,89,84.441891,3.9,3.700263,3.000201,3.648562,1.638504
219,David Silva,Manchester City,74,19.148936,90,85.320911,4.7,4.455648,4.546005,4.061553,1.637929


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,89,21.967213,134,125.692885,6.1,5.721840,8.443553,5.228147,1.917965
278,Pukki,Norwich City,65,25.208333,121,123.171731,4.8,4.886151,5.387560,5.223087,1.916506
409,Jiménez,Wolves,78,26.078431,133,131.623462,5.1,5.047215,4.534930,4.825163,1.807193
460,Abraham,Chelsea,77,24.074074,130,120.252885,5.4,4.995120,4.096681,3.962885,1.800116
166,Vardy,Leicester City,97,24.062500,154,131.743462,6.4,5.475053,2.555138,3.102193,1.757934
313,Ings,Southampton,71,25.961538,135,120.910577,5.2,4.657296,3.184266,4.194475,1.747852
187,Firmino,Liverpool,97,25.833333,124,131.684615,4.8,5.097469,6.279391,6.031155,1.636699
362,Deeney,Watford,63,14.864865,55,60.470577,3.7,4.068021,4.046070,4.389703,1.620738
502,Maupay,Brighton,58,25.142857,88,95.741154,3.5,3.807887,3.067335,2.869537,1.581139


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,128,23.013699,168,162.287922,7.3,7.051797,7.683430,7.778164,1.971037
215,De Bruyne,Manchester City,108,25.000000,175,156.276740,7.0,6.251070,6.943261,6.144492,1.902140
192,Mané,Liverpool,122,23.030303,152,139.255400,6.6,6.046616,2.571443,4.236628,1.731141
233,Rashford,Manchester Utd,89,21.967213,134,125.692885,6.1,5.721840,8.443553,5.228147,1.917965
182,Alexander-Arnold,Liverpool,77,25.932203,153,144.130662,5.9,5.557980,5.388696,5.343795,2.002957
214,Sterling,Manchester City,117,23.200000,116,128.701291,5.0,5.547469,4.819243,4.496669,1.621818
166,Vardy,Leicester City,97,24.062500,154,131.743462,6.4,5.475053,2.555138,3.102193,1.757934
210,Agüero,Manchester City,121,19.838710,123,104.182308,6.2,5.251466,5.022605,5.091143,1.509690
342,Son,Tottenham,100,21.034483,122,107.862429,5.8,5.127886,6.934087,5.706116,1.621580


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,128,23.013699,168,162.287922,7.3,7.051797,7.683430,7.778164,1.971037
103,Alonso,Chelsea,60,7.966102,47,35.066866,5.9,4.402011,NaN,6.851449,1.797113
215,De Bruyne,Manchester City,108,25.000000,175,156.276740,7.0,6.251070,6.943261,6.144492,1.902140
187,Firmino,Liverpool,97,25.833333,124,131.684615,4.8,5.097469,6.279391,6.031155,1.636699
150,Richarlison,Everton,82,24.117647,123,107.662706,5.1,4.464063,6.781432,5.981568,1.558919
340,Lloris,Tottenham,53,10.000000,42,43.978352,4.2,4.397835,5.706884,5.717816,1.910298
105,Azpilicueta,Chelsea,58,23.939394,79,105.499730,3.3,4.406951,4.612068,5.708263,1.829887
342,Son,Tottenham,100,21.034483,122,107.862429,5.8,5.127886,6.934087,5.706116,1.621580
447,Pieters,Burnley,42,15.000000,42,44.855800,2.8,2.990387,NaN,5.522582,1.459159
